In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [4]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

# 2 custom layer

In [44]:
keras.backend.variable?

In [10]:
np.random.random_sample?

In [12]:
np.ones?

In [13]:
keras.backend.dot?

In [83]:
class CustomDense(keras.layers.Layer):
    def __init__(self,unit,**kwargs):
        super(CustomDense,self).__init__(**kwargs)
        self.unit = unit
    
    def build(self,input_shape):
        '''
        第一次看到数据时，调用，并分配变量
        '''
        weight = np.random.random_sample(size=(input_shape[-1],self.unit))
        self.w = keras.backend.variable(value=weight,dtype=np.float32,name=self.name+'_weight')
        bias = np.ones(shape=self.unit)
        self.b = keras.backend.variable(value=bias,name=self.name+'_bias',dtype=np.float32)
        self.trainable_weights = [self.w,self.b]
    
    def call(self,input_x):
        self.add_loss(input_x*2)
        res = keras.backend.dot(input_x,self.w)+self.b
        return res

In [84]:
from keras.models import Sequential

In [85]:
nn = Sequential()
nn.add(keras.layers.Dense(units=5,input_shape=(2,)))
nn.add(CustomDense(unit=5))

In [86]:
nn.trainable_weights

[<tf.Variable 'dense_11/kernel:0' shape=(2, 5) dtype=float32_ref>,
 <tf.Variable 'dense_11/bias:0' shape=(5,) dtype=float32_ref>,
 <tf.Variable 'custom_dense_15/custom_dense_15_weight:0' shape=(5, 5) dtype=float32_ref>,
 <tf.Variable 'custom_dense_15/custom_dense_15_bias:0' shape=(5,) dtype=float32_ref>]

In [87]:
x = tf.ones(shape=(1,5))
cdense = CustomDense(5)

In [88]:
cdense(x)

<tf.Tensor 'custom_dense_16/add:0' shape=(1, 5) dtype=float32>

In [89]:
cdense.weights

[<tf.Variable 'custom_dense_16/custom_dense_16_weight:0' shape=(5, 5) dtype=float32_ref>,
 <tf.Variable 'custom_dense_16/custom_dense_16_bias:0' shape=(5,) dtype=float32_ref>]

In [90]:
cdense.trainable

True

In [80]:
cdense.losses

[<tf.Tensor 'custom_dense_14/mul:0' shape=(1, 5) dtype=float32>]

In [92]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# 3 custom model

In [96]:
class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed